In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import bootstrap
from tqdm import tqdm
import wandb
import tempfile
import os

tqdm.pandas()

In [2]:
api = wandb.Api()

In [3]:
# all old_bench
run_name_to_id = {
    "retmass_preconly": "1k6bzqu8",
    "retmass_fp": "wihjpo5l",
    "retmass_gnn": "ow2np6w0",
    "retform_preconly": "b48mjrk4",
    "retform_fp": "a9o2s4dn",
    "retform_gnn": "xvx3i4d1",
}

metric_cols = [
    "cos_sim",
    "cos_sim_sqrt",
    "js_sim",
    "test_hit_rate@1",
    "test_hit_rate@5",
    "test_hit_rate@20"
]

In [4]:
run_name_to_df = {}
with tempfile.TemporaryDirectory() as tmp_dp:
    for run_name, run_id in run_name_to_id.items():
        run = api.run(f"adamoyoung/msg/{run_id}")
        df_file = run.file("df_test.pkl")
        df_fn = df_file.name
        df_fp = os.path.join(tmp_dp,df_fn)
        df_file.download(replace=True, root=tmp_dp)
        run_name_to_df[run_name] = pd.read_pickle(df_fp)

In [5]:
run_name_to_df["retform_preconly"].mean()

losses              0.845981
cos_sim             0.154019
js_sim              0.408519
cos_sim_sqrt        0.165258
cos_sim_obj         0.154019
test_hit_rate@1     0.020896
test_hit_rate@5     0.085192
test_hit_rate@20    0.226542
dtype: float64

In [6]:
# put all dfs into one big df
df = []
for method, method_df in run_name_to_df.items():
    method_df["method"] = method
    df.append(method_df)
df = pd.concat(df)
df

,losses,cos_sim,js_sim,cos_sim_sqrt,cos_sim_obj,test_hit_rate@1,test_hit_rate@5,test_hit_rate@20,method
0,1.000000,0.000000,0.306853,0.000000,0.000000,0.0,0.0,0.0,retmass_preconly
1,1.000000,0.000000,0.306853,0.000000,0.000000,0.0,0.0,0.0,retmass_preconly
2,1.000000,0.000000,0.306853,0.000000,0.000000,0.0,0.0,0.0,retmass_preconly
3,1.000000,0.000000,0.306853,0.000000,0.000000,0.0,0.0,0.0,retmass_preconly
4,0.139411,0.860589,0.802057,0.730126,0.860589,0.0,1.0,1.0,retmass_preconly
...,...,...,...,...,...,...,...,...,...
9949,0.759845,0.204946,0.437068,0.240155,0.240155,0.0,0.0,1.0,retform_gnn
9950,0.982515,0.000745,0.317624,0.017485,0.017485,0.0,0.0,0.0,retform_gnn
9951,0.817273,0.038494,0.385755,0.182727,0.182727,0.0,0.0,0.0,retform_gnn
9952,1.000000,0.000000,0.306853,0.000000,0.000000,0.0,0.0,0.0,retform_gnn


In [7]:
def calculate_ci(df, metric_cols):

    # Calculate means for all metrics into a single table
    df_mean = df.groupby('method')[metric_cols].mean().round(3)

    # Calculate confidence intervals for all metrics into a single table
    def get_ci(col_vals, confidence_level=0.999, n_resamples=20_000):
        res = bootstrap((col_vals,), np.mean, confidence_level=confidence_level, n_resamples=n_resamples)
        ci = res.confidence_interval
        return f'{ci.low:.3f}-{ci.high:.3f}'
    def get_ci_for_each_col(df_method):
        return df_method.apply(get_ci, axis=0)
    df_ci = df.groupby('method')[metric_cols].progress_apply(lambda df_method: get_ci_for_each_col(df_method))

    # Merge tables with means and confidence intervals
    for col in metric_cols:
        df_mean[col] = df_mean[col].astype(str) + ' (' + df_ci[col] + ')'
    return df_mean

In [8]:
df_ci = calculate_ci(df, metric_cols)

  0%|          | 0/6 [00:00<?, ?it/s]

100%|██████████| 6/6 [03:22<00:00, 33.81s/it]


In [9]:
df_ci

,cos_sim,cos_sim_sqrt,js_sim,test_hit_rate@1,test_hit_rate@5,test_hit_rate@20
method,,,,,,
retform_fp,0.244 (0.233-0.256),0.268 (0.259-0.277),0.475 (0.469-0.481),0.076 (0.068-0.085),0.227 (0.213-0.241),0.441 (0.425-0.458)
retform_gnn,0.19 (0.181-0.200),0.22 (0.212-0.229),0.443 (0.437-0.448),0.036 (0.030-0.043),0.136 (0.124-0.146),0.338 (0.322-0.353)
retform_preconly,0.154 (0.145-0.165),0.165 (0.156-0.174),0.409 (0.403-0.415),0.021 (0.017-0.026),0.085 (0.076-0.095),0.227 (0.214-0.240)
retmass_fp,0.249 (0.239-0.261),0.273 (0.264-0.282),0.478 (0.472-0.484),0.084 (0.076-0.094),0.214 (0.200-0.228),0.386 (0.370-0.402)
retmass_gnn,0.192 (0.182-0.203),0.218 (0.210-0.226),0.442 (0.436-0.447),0.039 (0.034-0.046),0.119 (0.109-0.130),0.263 (0.248-0.278)
retmass_preconly,0.154 (0.144-0.165),0.165 (0.156-0.174),0.409 (0.403-0.415),0.004 (0.002-0.006),0.017 (0.013-0.022),0.072 (0.064-0.080)
